In [1]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/Y8w3wHsEdYQSZRp/download', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'])
df=pd.read_csv('positivos_covid.csv', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'])
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
fecha_corte.to_json("../resultados/fecha_corte_positivos.json")

In [2]:
# DIARIO AG, PCR, PR
df_positivos=df[['FECHA_RESULTADO','METODODX','SEXO']].groupby(['FECHA_RESULTADO','METODODX']).count()
df_positivos=df_positivos.reset_index()
df_positivos=df_positivos.pivot(index='FECHA_RESULTADO', columns='METODODX', values='SEXO')
df_positivos=df_positivos.rename_axis(None, axis=1)
df_positivos=df_positivos.fillna(0).astype('int')
df_positivos

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,5,0
2020-03-08,0,2,0
2020-03-09,0,3,0
2020-03-10,0,1,0
...,...,...,...
2021-05-18,2887,1702,661
2021-05-19,2738,1413,545
2021-05-20,2463,1349,474


In [3]:
# ACUMULADO AG, PCR, PR
df_positivos_cum=df_positivos.cumsum()
df_positivos_cum

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,6,0
2020-03-08,0,8,0
2020-03-09,0,11,0
2020-03-10,0,12,0
...,...,...,...
2021-05-18,423765,570708,913930
2021-05-19,426503,572121,914475
2021-05-20,428966,573470,914949


In [4]:
# ACUMULADO POR DEPARTAMENTO
df_positivos_departamento=df[['DEPARTAMENTO','METODODX','SEXO']].groupby(['DEPARTAMENTO', 'METODODX']).count()
df_positivos_departamento=df_positivos_departamento.reset_index()
df_positivos_departamento=df_positivos_departamento.pivot(index='DEPARTAMENTO', columns='METODODX', values='SEXO')
df_positivos_departamento

METODODX,AG,PCR,PR
DEPARTAMENTO,,,
AMAZONAS,5629,4437,18108
ANCASH,25924,14710,27744
APURIMAC,8998,5125,8346
AREQUIPA,15795,18055,46937
AYACUCHO,7145,6399,14848
CAJAMARCA,16539,13475,27224
CALLAO,14957,43618,35894
CUSCO,20771,16312,23905
HUANCAVELICA,4394,1941,8072


In [6]:
# ACUMULADO POR GRUPO ETARIO
bins = [18,20,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 19 años','20 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [900000,5700000,5300000,4400000,3300000,2300000,1300000,700000] 
df_positivos_edades = df
df_positivos_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_positivos_edades = df_positivos_edades.groupby(['GRUPO_ETARIO'])["METODODX"].count().reset_index()
df_positivos_edades.rename(columns = {'METODODX':'POSITIVOS'}, inplace = True)
df_positivos_edades['POBLACION']=poblacion_por_grupo_etario
df_positivos_edades['PORCENTAJE']=round(df_positivos_edades['POSITIVOS']/df_positivos_edades['POBLACION']*100,2)
df_positivos_edades=df_positivos_edades.set_index('GRUPO_ETARIO')
df_positivos_edades

,POSITIVOS,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 19 años,34642,900000,3.85
20 a 29 años,348473,5700000,6.11
30 a 39 años,427067,5300000,8.06
40 a 49 años,366094,4400000,8.32
50 a 59 años,294182,3300000,8.91
60 a 69 años,192063,2300000,8.35
70 a 79 años,100489,1300000,7.73
80 años a más,49665,700000,7.10


In [7]:
df_positivos.to_csv('../resultados/positivos_diarios.csv')
df_positivos_cum.to_csv('../resultados/positivos_acumulados.csv')
df_positivos_departamento.to_csv('../resultados/positivos_por_departamentos.csv')
df_positivos_edades.to_csv('../resultados/positivos_por_edades.csv')